In [2]:
import numpy as np # library to handle data in a vectorized manner
!pip install geocoder
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
!pip install folium
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 98 kB 3.4 MB/s 
Libraries imported.


### **Getting the Data**

Build a dataframe of neighborhoods in Hyderabad, India by web scraping the data from Wikipedia page Get the geographical coordinates of the neighborhoods by Python Geocoder package Obtain the venue data for the neighborhoods from Foursquare API Explore and cluster the neighbourhoods Select the best cluster to open a new shopping mall

### **Business Problem**

This project is mainly focused on geospatial analysis of the Hyderabad City to understand which would be the best place to open a new mall

In [13]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text

In [14]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')


In [15]:
# create a list to store neighborhood data
neighborhoodList = []

In [16]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [17]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"


In [18]:
# print the number of rows of the dataframe
kl_df.shape

(143, 1)

In [19]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [20]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [21]:
coords

[[17.394870000000026, 78.47076000000004],
 [17.394870000000026, 78.47076000000004],
 [17.560190000000034, 78.48710000000005],
 [17.394870000000026, 78.47076000000004],
 [17.38116388383898, 78.42949474001429],
 [17.46661000000006, 78.36321000000004],
 [17.297960000000046, 78.60632000000004],
 [17.394870000000026, 78.47076000000004],
 [12.963480000000061, 77.61297000000008],
 [13.027180000000044, 77.65045000000003],
 [13.131820000000062, 77.67001000000005],
 [26.85442000000006, 75.75681000000003],
 [12.98929000000004, 77.65111000000007],
 [12.98929000000004, 77.65111000000007],
 [12.922310000000039, 77.56988000000007],
 [17.372920000000022, 78.44078000000007],
 [12.938980000000072, 77.57137000000006],
 [17.493630000000053, 78.43412000000006],
 [12.931350000000066, 77.68468000000007],
 [17.394870000000026, 78.47076000000004],
 [17.465080000000057, 78.42506000000003],
 [12.90008000000006, 77.60432000000003],
 [13.086400000000026, 77.41036000000008],
 [12.983991999946278, 77.6404896228702],

In [22]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [23]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [24]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df.head()

(143, 3)


,Neighborhood,Latitude,Longitude
0,List of areas in Bangalore Cantonment,17.394870,78.470760
1,List of areas in Bengaluru Pete,17.394870,78.470760
2,List of neighbourhoods in Bangalore,17.560190,78.487100
3,Adugodi,17.394870,78.470760
4,"Agara, Bangalore",17.381164,78.429495


In [25]:
# get the coordinates of Bengaluru
address = 'Bengaluru, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru, India 12.9791198, 77.5912997.


In [27]:
# create map of Bengaluru using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [28]:
# save the map as HTML file
map_kl.save('/content/drive/MyDrive/Colab Notebooks/Neighbourhood-Segmentation/map_kl.html')

## **Use the foursquare API to explore the neighbourhoods**

In [29]:
CLIENT_ID = 'RJ3CLZG4JCDCVREY5S2R1WFPNX0A03MYJRD4CTPAWWBQJW2K' # your Foursquare ID
CLIENT_SECRET = 'TJ4NPL40BJWU4A51PSHNXVK2ADD2FZRPANQWRNONGGDAMCYN' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RJ3CLZG4JCDCVREY5S2R1WFPNX0A03MYJRD4CTPAWWBQJW2K
CLIENT_SECRET:TJ4NPL40BJWU4A51PSHNXVK2ADD2FZRPANQWRNONGGDAMCYN


In [30]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [31]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(7575, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of areas in Bangalore Cantonment,17.39487,78.47076,Subhan Bakery,17.392412,78.464712,Bakery
1,List of areas in Bangalore Cantonment,17.39487,78.47076,Nizam club,17.403221,78.468729,Lounge
2,List of areas in Bangalore Cantonment,17.39487,78.47076,Cafe Niloufer & Bakers,17.399715,78.462881,Café
3,List of areas in Bangalore Cantonment,17.39487,78.47076,Cafe Bahar,17.399595,78.478566,Indian Restaurant
4,List of areas in Bangalore Cantonment,17.39487,78.47076,Mayur Pan Shop,17.388894,78.480578,Juice Bar
5,List of areas in Bangalore Cantonment,17.39487,78.47076,Lal Bahadur Shastri Stadium,17.399140,78.472666,Stadium
6,List of areas in Bangalore Cantonment,17.39487,78.47076,Pragati,17.388088,78.481134,South Indian Restaurant
7,List of areas in Bangalore Cantonment,17.39487,78.47076,Chicha's,17.403255,78.460152,Hyderabadi Restaurant
8,List of areas in Bangalore Cantonment,17.39487,78.47076,Karachi Bakery,17.383454,78.475075,Bakery
9,List of areas in Bangalore Cantonment,17.39487,78.47076,Birla Science Museum,17.403854,78.469457,Science Museum


### Lets check how many venues were returned for each neighbourhood

In [32]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,96,96,96,96,96,96
"Agara, Bangalore",23,23,23,23,23,23
Ananthnagar,44,44,44,44,44,44
Arekere,96,96,96,96,96,96
Austin Town,100,100,100,100,100,100
BTM Layout,96,96,96,96,96,96
Babusapalya,50,50,50,50,50,50
"Bagalur, Bangalore Urban",3,3,3,3,3,3
Bahubalinagar,4,4,4,4,4,4


In [ ]:
#venues
#venues_df[venues_df['Neighborhood']=='Sikh Village'].groupby

### **Let's find out how many unique categories can be curated from all the returned venues**

In [33]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 217 uniques categories.


In [34]:
venues_df['VenueCategory'].unique()[:50] #displays all the 217 category names

array(['Bakery', 'Lounge', 'Café', 'Indian Restaurant', 'Juice Bar',
       'Stadium', 'South Indian Restaurant', 'Hyderabadi Restaurant',
       'Science Museum', 'Ice Cream Shop', 'Food Truck', 'Hotel',
       'Neighborhood', 'Burger Joint', 'Hotel Bar', 'Dessert Shop',
       'Diner', 'Shoe Store', 'Smoke Shop', 'Chinese Restaurant',
       'Chaat Place', 'Lake', 'Mobile Phone Shop', 'Bar',
       'Breakfast Spot', 'Restaurant', 'Middle Eastern Restaurant',
       'Vegetarian / Vegan Restaurant', 'Fast Food Restaurant',
       'Performing Arts Venue', 'Coffee Shop', 'Gaming Cafe', 'Multiplex',
       'Department Store', 'Shopping Mall', 'Fried Chicken Joint', 'Park',
       'Clothing Store', 'Pizza Place', 'Hookah Bar', 'Farmers Market',
       'Snack Place', 'Indie Movie Theater', 'Jewelry Store',
       'Food Court', 'Sandwich Place', "Men's Store",
       'Arts & Crafts Store', 'Sports Bar', 'Falafel Restaurant'],
      dtype=object)

In [35]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### **Analyze each neighbourhood**

In [36]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head(20)

(7575, 218)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Chaat Place,Cheese Shop,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Lab,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo
0,List of areas in Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,List of areas in Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).sum().reset_index()
#df.group/by("state")["last_name"].count()

#kl_onehot.head()
print(kl_grouped.shape)
kl_grouped

(137, 218)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Chaat Place,Cheese Shop,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Lab,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo
0,Adugodi,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,4,0,1,0,0,4,0,1,0,3,0,0,0,0,0,0,0,0,0,0,2,3,0,1,0,0,0,0,0,0,1,5,0,0,0,0,1,0,0,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5,1,0,2,4,0,14,0,1,0,0,0,0,1,2,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,2,0,1,0,1,0,1,1,1,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,"Agara, Bangalore",0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [38]:
#a=kl_grouped['Neighborhoods'][0]


In [39]:
len((kl_grouped[kl_grouped["Shopping Mall"] > 0]))  #So there are 75 shopping malls in Bengaluru(which it very high)
#So now we want to select a good location where the no of shopping malls are less so that our chances of setting up a shopping mall at that location  should be good

75

## **Create a dataframe for shopping mall data only**

In [40]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [41]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Adugodi,1
1,"Agara, Bangalore",1
2,Ananthnagar,0
3,Arekere,1
4,Austin Town,1


## **Now cluster the neighbourhoods**

Run k-means to cluster the neighborhoods in Hyderabad into 3 clusters.

In [42]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 1, 1, 1, 2, 2, 1, 1], dtype=int32)

In [43]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [44]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head(10)

,Neighborhood,Shopping Mall,Cluster Labels
0,Adugodi,1,1
1,"Agara, Bangalore",1,1
2,Ananthnagar,0,2
3,Arekere,1,1
4,Austin Town,1,1
5,BTM Layout,1,1
6,Babusapalya,0,2
7,"Bagalur, Bangalore Urban",0,2
8,Bahubalinagar,1,1
9,Baiyyappanahalli,1,1


In [45]:
#Add latitude and longitude values by using the join operation(the new dataframe with the old dataframe containing the latitude and longitude values)
#kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

#print(kl_merged.shape)
kl_merged['Latitude'] = df_coords['Latitude']
kl_merged['Longitude'] = df_coords['Longitude']
kl_merged.head() # check the last columns!

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adugodi,1,1,17.394870,78.470760
1,"Agara, Bangalore",1,1,17.394870,78.470760
2,Ananthnagar,0,2,17.560190,78.487100
3,Arekere,1,1,17.394870,78.470760
4,Austin Town,1,1,17.381164,78.429495


In [46]:
# sorting the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(137, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
81,Lingarajapuram,2,0,12.882290,77.569230
116,"Siddapura, Bangalore",2,0,17.449060,78.419150
49,Gowdanapalya,2,0,17.449590,78.420540
40,"Fraser Town, Bangalore",2,0,17.400870,78.521870
41,G. M. Palya,2,0,17.394870,78.470760
110,Richmond Town,2,0,13.117880,77.559210
108,Ramamurthy Nagar,2,0,17.470000,78.352720
127,Vasanth Nagar,3,0,13.173520,77.118990
46,Garudacharpalya,2,0,17.364610,78.456810
78,Krishnarajapuram,2,0,17.394870,78.470760


In [47]:
kl_merged['Shopping Mall'].max()

3

# **Now we visualize the resulting clusters**

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
# save the map as HTML file
map_clusters.save('/content/drive/MyDrive/Colab Notebooks/Neighbourhood-Segmentation.html')

## **Examine clusters**

In [50]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 0])# -132 neighbourhoods/places in this cluster 0

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
81,Lingarajapuram,2,0,12.88229,77.56923
116,"Siddapura, Bangalore",2,0,17.44906,78.41915
49,Gowdanapalya,2,0,17.44959,78.42054
40,"Fraser Town, Bangalore",2,0,17.40087,78.52187
41,G. M. Palya,2,0,17.39487,78.47076
110,Richmond Town,2,0,13.11788,77.55921
108,Ramamurthy Nagar,2,0,17.47000,78.35272
127,Vasanth Nagar,3,0,13.17352,77.11899
46,Garudacharpalya,2,0,17.36461,78.45681
78,Krishnarajapuram,2,0,17.39487,78.47076


In [51]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 1])
#There are 51 neighbourhoods/places in cluster 1 which is the highest among the 3 clusters

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
72,Kengeri,1,1,12.965530,77.666660
76,Koppa gate,1,1,17.408370,78.594640
135,Yelahanka,1,1,13.076000,77.652400
67,Kalasipalyam,1,1,12.936560,77.585220
80,Kundalahalli,1,1,12.966210,77.649800
71,Kammanahalli,1,1,17.394870,78.470760
0,Adugodi,1,1,17.394870,78.470760
63,Jeevanbimanagar,1,1,17.394870,78.470760
60,Jakkur,1,1,17.394870,78.470760
59,Ittamadu,1,1,17.394870,78.470760


In [52]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2] 
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 2] ) #-There are only 9 shopping malls in the cluster which is the least among the 3 clusters

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
2,Ananthnagar,0,2,17.560190,78.487100
109,Ramanjaneyanagar,0,2,17.511610,78.294110
6,Babusapalya,0,2,17.297960,78.606320
107,Ramagondanahalli,0,2,17.452140,78.546060
106,Ramachandrapura,0,2,17.498060,78.407367
104,Rajajinagar,0,2,17.394870,78.470760
19,Binnamangala Manavarthe Kaval,0,2,17.394870,78.470760
133,Wilson Garden,0,2,17.494980,78.384460
105,"Rajarajeshwari Nagar, Bangalore",0,2,12.894340,77.585070
113,Sanjaynagar,0,2,12.962330,77.601230


# **Final observation**

A good number of shopping malls are concentrated in the central area of Bengaluru city, with the highest number in cluster 0 and moderate number in cluster 1. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighbourhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighbourhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighbourhoods in cluster 0 which already have a high concentration of shopping malls and suffering from intense competition.


